In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm
# GPU 사용
device = torch.device("cuda")

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator").to(device)
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/CP1/CP1_koelectra/model(nsmc_aihub6).pt'
model.load_state_dict(torch.load(path))

In [ ]:
def predict(text):
    encoded_review = tokenizer.encode_plus(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=128,
        padding=True,
        add_special_tokens=True
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output[0], dim=1)

    # print('텍스트: ' + text)
    if str(prediction) == "tensor([1], device='cuda:0')":
        return '긍정'
    else:
        return '부정'

In [ ]:
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
from datetime import datetime, timedelta, timezone

client = MongoClient(
    host='', 
    port=4001,
    username='user',
    password='appleBanana',
    authSource='news')
db = client['news']

In [ ]:
# cnt = 0
# arr = []
# for i in db['naver'].find({'date': {
#     '$gte': datetime.strptime('2021-01-01T00:00:00+0900', '%Y-%m-%dT%H:%M:%S%z'),
#     '$lt': datetime.strptime('2021-02-01T00:00:00+0900', '%Y-%m-%dT%H:%M:%S%z')
#     }}, batch_size=100) :
#     cnt += 1
#     i['sentiment_result'] = predict(i['title'])

#     for comment in i['comments'] :
#         comment['sentiment_result'] = predict(comment['text'])
 
#     arr.append(i)

#     if cnt % 100 == 0 :
#         print(cnt)
#         try :
#             db['naver_21'].insert_many(arr, ordered=False)
#         except :
#             pass
#         arr = []
# if arr :
#     try :
#         db['naver_21'].insert_many(arr,  ordered=False)
#     except  :
#         pass

In [ ]:
data = list(db['naver'].find({'date': {
    '$gte': datetime.strptime('2021-04-01T00:00:00+0900', '%Y-%m-%dT%H:%M:%S%z'),
    '$lt': datetime.strptime('2021-05-01T00:00:00+0900', '%Y-%m-%dT%H:%M:%S%z')
    }}, batch_size=500))

In [ ]:
len(data)

In [ ]:
# data1 = data[6530:35000]

In [ ]:
cnt = 0
arr = []
for i in tqdm(data):
    cnt += 1
    i['sentiment_result'] = predict(i['title'])

    for comment in i['comments'] :
        comment['sentiment_result'] = predict(comment['text'])
 
    arr.append(i)

    if cnt % 100 == 0 :
        print(cnt)
        try :
            db['naver_20'].insert_many(arr, ordered=False)
        except :
            pass
        arr = []
if arr :
    try :
        db['naver_20'].insert_many(arr,  ordered=False)
    except  :
        pass